In [1]:
import pandas as pd
import numpy as np
import os
from pcntoolkit.dataio.fileio import load as ptkload
from pcntoolkit.dataio.fileio import save as ptksave

import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
import matplotlib as mpl
from pathlib  import Path
import matplotlib.ticker as ticker

import seaborn as sns

sns.set_style("whitegrid")
sns.set_palette("husl", 9)

# globals
root_dir = '/project_cephfs/3022017.06/ENIGMA_ANX/'
proc_dir = os.path.join(root_dir,'Z_stat/')
data_dir = os.path.join(proc_dir,'data/')
w_dir = os.path.join(proc_dir,'vox/')
save_dir = os.path.join(w_dir,'Validation/')
mask_nii = ('/opt/fmriprep/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_mask.nii.gz')
ex_nii = os.path.join(data_dir, 'ENIGMA_FC_tr_1.nii.gz')

/home/preclineu/hansav/.conda/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load in the Z_est files
Z_est_control_test = ptkload(os.path.join(w_dir,'Z_estimate.pkl'), mask=mask_nii)
Z_est_clinical = ptkload(os.path.join(w_dir,'Z_predcl.pkl'), mask=mask_nii)
Full_sample_deviations = np.append(Z_est_control_test,Z_est_clinical, axis = 0)

#Load in the contingency awareness data
Measures = pd.read_csv('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/data/all_test_validation.csv', usecols = ["Group_Dataset", 
                                                                                                                "Anxiety_instrument",
                                                                                                                "Anxiety_score",
                                                                                                                "Depression_instrument",
                                                                                                                "Depression_score", 
                                                                                                                'Principal_diagnosis_current']) 
Measures.replace(to_replace='NA/does not apply', value='NA', regex=True, inplace=True)

Measures['Anxiety_score'] = pd.to_numeric(Measures['Anxiety_score'], errors='coerce').astype('Int64')
Measures['Depression_score'] = pd.to_numeric(Measures['Depression_score'], errors='coerce').astype('Int64')


# FOR STAI- SPANISH VERSION: add 20 to make scale the same as others:
spanish_STAI_sites = ["Barcelona_Cardoner", "Barcelona_Soriano_dataset_1", "Barcelona_Soriano_dataset_2"]
# Check if instrument_value is 1 and the Group_Dataset is in the specified list
for index, row in Measures.iterrows():
    if row['Group_Dataset'] in spanish_STAI_sites and Measures.at[index, 'Anxiety_instrument'] == 'stai-trait':
        # Modify Anxiety_score by adding 20
        Measures.at[index, 'Anxiety_score'] += 20


#Mask by participants for whom CA data is available
#mask_Anx = Measures["Anxiety_score"].notna() #remove NAs
#mask_select_measure = Measures['Anxiety_instrument'].isin(['stai-trait']) #CHANGE THIS TO QUESTIONNIARE OF INTEREST
#beck anxiety inventory
#hamilton anxiety
#other (scared total score)
#other (staic)
#stai-state
#stai-trait

mask_exclude_diagnosis = ~Measures['Principal_diagnosis_current'].isin(['others', 'schizophrenia']) #and remove others and schizophrenia
combined_mask_Anx = mask_Anx & mask_exclude_diagnosis 
combined_mask_Anx = combined_mask_Anx & mask_select_measure

#Anx_sample = Measures['Anxiety_score'][combined_mask_Anx].to_numpy()
#Anx_sample_deviations = Full_sample_deviations[combined_mask_Anx]
#print('Anxiety data available for: '+str(len(Anx_sample)) +' people')
#print(Anx_sample)

mask_Dep = Measures["Depression_score"].notna() #remove NAs
mask_select_measure = Measures['Depression_instrument'].isin(['beck depression inventory']) #CHANGE THIS TO QUESTIONNIARE OF INTEREST
combined_mask_Dep = mask_Dep & mask_exclude_diagnosis
combined_mask_Dep = combined_mask_Dep & mask_select_measure

Dep_sample = Measures['Depression_score'][combined_mask_Dep].to_numpy()
Dep_sample_deviations = Full_sample_deviations[combined_mask_Dep]
print('Depression data available for: '+str(len(Dep_sample)) +' people')
print(Dep_sample)

Depression data available for: 152 people
[3 2 10 2 0 8 1 11 2 1 5 3 7 11 9 9 12 9 2 11 21 0 19 31 45 20 24 31 41 13
 17 24 24 35 32 46 10 14 25 43 24 18 27 37 23 14 23 11 20 7 9 5 1 3 0 6 1
 5 0 0 1 7 0 2 1 1 3 16 0 0 0 0 5 0 1 0 0 1 0 2 5 0 5 0 1 0 1 4 7 0 2 0 0
 7 36 20 20 13 44 13 6 39 9 16 20 8 34 17 32 23 34 23 0 7 29 21 22 17 20
 21 7 3 3 19 16 26 6 11 9 21 31 9 1 26 41 16 28 24 18 23 37 14 27 19 18 0
 4 6 20 32 13 12]


In [5]:
#Define parameters
X1 = Dep_sample_deviations #Deviations
y = Dep_sample.ravel()
n_samples, n_features = X1.shape
random_state = np.random.RandomState(0)


In [6]:
#%%ELASTIC NET CV

from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

from sklearn.model_selection import train_test_split


model_alpha = []
model_intercept = []

ypred_cf = []
score_cf = []
ev_cf = []
mse_cf = []
coefs_cf = []

for i in range(0,10):
    print('Iteration',i)

    alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]
    xtrain, xtest, ytrain, ytest = train_test_split(X1, y, test_size=0.1)
    
    #ElasticNetCV is a cross-validation class that can search multiple alpha values 
    #and applies the best one. We'll define the model with alphas value and fit 
    #it with xtrain and ytrain data.
    
    elastic_cv=ElasticNetCV(alphas=alphas, cv=5)
    model = elastic_cv.fit(xtrain, ytrain)
    
    print(model.alpha_)
    model_alpha.append(model.alpha_)
    print(model.intercept_)
    model_intercept.append(model.intercept_)
    
    ypred = model.predict(xtest)
    score = model.score(xtest, ytest)
    ev = explained_variance_score(ytest, ypred)
    mse = mean_squared_error(ytest, ypred)
    print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
          .format(score, mse, np.sqrt(mse)))
    
    
    ypred_cf.append(ypred)
    score_cf.append(score)
    ev_cf.append(ev)
    mse_cf.append(mse)
    coefs_cf.append(model.coef_)
    
combined_elastic_net = pd.DataFrame(list(zip(score_cf, ev_cf, mse_cf, model_intercept)),
                                    columns=['R2','EV', 'MSE', 'Model_Intercept']) 
combined_elastic_net.to_csv(os.path.join(save_dir,('Depression_BDI_Elastic_Net.csv')))


mean_coefs = np.mean(coefs_cf, axis = 0)
ptksave(mean_coefs, os.path.join(save_dir,('Depression_BDI_mean.nii.gz')), example=ex_nii, mask=mask_nii)


median_coefs = np.median(coefs_cf, axis = 0)
ptksave(median_coefs, os.path.join(save_dir,('Depression_BDI_median.nii.gz')), example=ex_nii, mask=mask_nii)


coefs_cf = np.array(coefs_cf)
freq_coefs = (coefs_cf > 0.0001).sum(axis=0) >= 5
ptksave(coefs_cf, os.path.join(save_dir,('Depression_BDI_coefs.nii.gz')), example=ex_nii, mask=mask_nii)

binary_mask = (coefs_cf > 0.005).sum(axis=0) >= 5
ptksave(binary_mask, os.path.join(save_dir,('Depression_BDI_gt5_mask.nii.gz')), example=ex_nii, mask=mask_nii)
count_array = (coefs_cf > 0.0001).sum(axis=0)
ptksave(count_array, os.path.join(save_dir,('Depression_BDI_count.nii.gz')), example=ex_nii, mask=mask_nii)



Iteration 0
1.0
12.750799103385464
R2:-0.210, MSE:203.38, RMSE:14.26
Iteration 1
1.0
13.27001860342568
R2:-0.118, MSE:103.51, RMSE:10.17
Iteration 2
1.0
13.111495577955168
R2:-0.266, MSE:191.50, RMSE:13.84
Iteration 3
1.0
13.0836246378012
R2:0.030, MSE:211.11, RMSE:14.53
Iteration 4
1.0
13.773250833183967
R2:-0.609, MSE:100.34, RMSE:10.02
Iteration 5
1.0
12.842968952696364
R2:-0.273, MSE:146.05, RMSE:12.09
Iteration 6
1.0
13.36276132901337
R2:-0.480, MSE:161.45, RMSE:12.71
Iteration 7
1.0
13.51145291579118
R2:0.131, MSE:105.11, RMSE:10.25
Iteration 8
1.0
13.346168140157216
R2:-0.934, MSE:150.08, RMSE:12.25
Iteration 9
1.0
12.7491870408895
R2:0.157, MSE:112.00, RMSE:10.58


In [ ]:
#%%
EPSILON = 1e-4  # This is to avoid division by zero while taking the base 10 logarithm
fpath = Path("/project_cephfs/3022017.06/ENIGMA_ANX/arial.ttf")

plt.figure()
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 5, 5
plt.semilogx(model.alphas_ + EPSILON, model.mse_path_, ':')
plt.plot(model.alphas_ + EPSILON, model.mse_path_.mean(axis=-1), 'k',
         label='Average across the folds', linewidth=2)
plt.axvline(model.alpha_ + EPSILON, linestyle='--', color='k',
            label=r'$\alpha$: CV')
plt.legend()
plt.yticks(font = fpath)
plt.xticks(size = 8, font = fpath)
plt.xlabel(r'$\alpha$', font = fpath)
plt.ylabel('Mean square error', font = fpath)
plt.title('Mean square error on each fold: coordinate descent ', font = fpath)
plt.axis('tight')
plt.show()
plt.savefig('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/plots/Depression_BDI_ElasticNetCV5.png', dpi=300)


In [ ]:
#%%PLOT ANXIETY SCORES
# Set up the font path
unique, counts = np.unique(y, return_counts=True)
# Assuming unique is a NumPy array or a list
unique = np.array(unique)

# Create an array of ones with the same length as unique
y_values = np.ones(len(unique))


fpath = Path("/project_cephfs/3022017.06/ENIGMA_ANX/arial.ttf")

# Set up the figure and axis
sns.set_style("white")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = 5, 1
fig, ax = plt.subplots()
# Plot the stacked bar chart
plt.ylim=(0.95, 1.05)
sns.scatterplot(x = unique, y = y_values, y_jitter = 10, size = (counts/sum(counts)*100), alpha = 0.5)# palette=dict('1'="#B7092C", '2'="#F7B192", '3'="#B9D0FA", '4'='#4052CB'), )


#Add line for clinical threshold
#Mapping: 
#                            {'NA' : 'NA', 
#                            'stai-trait' : 40,  20-80
#                            'other (masq - anxious arousal)' : ,  
#                            'stai-state' : ,
#                            'hamilton anxiety': 17, 0-56
#                            'other (scared total score)' : 25,  0-82
#                            'other (dass-21_anxiety subscale)': 
#                            'other (staic)': , 
#                            'beck anxiety inventory': , 
#                            'others': }  

plt.axvline(x = 17,    # Line on x = clinical cut off
           ymin = 0, # Bottom of the plot
           ymax = 1, 
           color = "#4052CB",
           linestyle = "--",
           linewidth = 0.75) # Top of the plot

# Add labels and title
plt.yticks(font = fpath)
plt.xticks(size = 8, font = fpath)
ax.set_xticks([0, 10, 20, 30, 40, 50])
#ax.set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80])
ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax.set_xlabel('Anxiety Score', font = fpath, size = 8)
sns.despine(top=True, left=True, bottom=False, right = True)

plt.legend(loc = 'best', prop=fpath, ncol=5)
#plt.legend().set_visible(False)

# Show the plot
plt.show()
fig.savefig('/project_cephfs/3022017.06/ENIGMA_ANX/Z_stat/plots/Depression_BDI_withlegend.png', dpi=300)